In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Input,SpatialDropout1D, Embedding, LSTM, Dense, merge, Convolution2D, Lambda, GRU, TimeDistributed, Reshape, Permute, Convolution1D, Masking, Bidirectional
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras import optimizers, callbacks

Using TensorFlow backend.


In [0]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
#tf.initialize_all_variables().run()

In [3]:
from tensorflow.python.client import device_lib
import gzip
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15654885223557092269
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3712327819531177089
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2143073228782865796
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14892338381
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14450658074724052095
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
# Install the PyDrive wrapper & import libraries.\n# This only needs to be done once per notebook.\n!pip install -U -q PyDrive\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n# Authenticate and create the PyDrive client.\n# This only needs to be done once per notebook.\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\n# Download a file based on its file ID.\n#\n# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz\nfile_id = \'REPLACE_WITH_YOUR_FILE_ID\'\ndownloaded = drive.CreateFile({\'id\': file_id})\nprint(\'Downloaded content "{}"\'.format(downloaded.GetContentString()))'

In [5]:
# !git clone https://github.com/pythonLoader/Lula_Dataset.git
# !git clone https://github.com/Arpita33/WeakILSDataset.git

!git clone https://github.com/pythonLoader/11-taxonDataset.git

fatal: destination path '11-taxonDataset' already exists and is not an empty directory.


In [0]:
import numpy as np

In [7]:
gt_strongILS_whole = np.load("11-taxonDataset/StrongILS/strong_ILS_valid_tree.npy")
gt_strongILS_taxa_removed = np.load("11-taxonDataset/StrongILS/strong_ILS_train_tree.npy")

gt_weakILS_whole = np.load("11-taxonDataset/WeakILS/weak_ILS_valid_tree.npy")
gt_weakILS_taxa_removed = np.load("11-taxonDataset/WeakILS/weak_ILS_train_tree_final.npy")

gt_total_whole = gt_strongILS_whole
gt_total_taxa_removed = gt_strongILS_taxa_removed
# gt_total_whole = gt_weakILS_whole
# gt_total_taxa_removed = gt_weakILS_taxa_removed


# gt_total_whole = np.concatenate((gt_strongILS_whole, gt_weakILS_whole), axis=0, out=None)
# gt_total_taxa_removed = np.concatenate((gt_strongILS_taxa_removed,gt_weakILS_taxa_removed),axis=0, out=None)

#gt_total_whole.shape

Train_data_X_arr = gt_total_taxa_removed
Train_data_Y_arr = gt_total_whole

#print(Train_data_Y_arr.shape)

print(Train_data_X_arr.shape)
print(Train_data_Y_arr.shape)

(3885, 165, 3)
(3885, 165, 3)


In [8]:
for i in range(Train_data_X_arr.shape[1]):

  print(f'{Train_data_X_arr[2][i]} and {Train_data_Y_arr[2][i]}')

[0. 0. 0.] and [0. 0. 1.]
[0. 0. 0.] and [0. 1. 0.]
[0. 0. 0.] and [0. 1. 0.]
[0. 0. 0.] and [0. 1. 0.]
[0. 0. 0.] and [1. 0. 0.]
[0. 0. 0.] and [1. 0. 0.]
[0. 0. 0.] and [1. 0. 0.]
[0. 0. 0.] and [1. 0. 0.]
[0. 0. 0.] and [1. 0. 0.]
[0. 1. 0.] and [0. 1. 0.]
[0. 1. 0.] and [0. 1. 0.]
[0. 1. 0.] and [0. 1. 0.]
[0. 1. 0.] and [0. 1. 0.]
[0. 0. 0.] and [0. 1. 0.]
[0. 1. 0.] and [0. 1. 0.]
[0. 0. 0.] and [0. 1. 0.]
[0. 1. 0.] and [0. 1. 0.]
[1. 0. 0.] and [1. 0. 0.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 0.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 0.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 0.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 0.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 0.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 0.] and [0. 0. 1.]
[0. 0. 1.] and [0. 0. 1.]
[0. 0. 0.] and [1. 0. 0.]
[1. 0. 0.] and [1. 0. 0.]
[0. 0. 0.] and [1. 0. 0.]
[1. 0. 0.] a

In [9]:
gene_tree_num = Train_data_X_arr.shape[0]
triplet = 3
print(gene_tree_num)

3885


In [10]:
x = Train_data_X_arr[0][:40]
y = Train_data_Y_arr[0][:40]

print(x)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [11]:
Train_data_X_reshaped = Train_data_X_arr.reshape(Train_data_X_arr.shape[1],Train_data_X_arr.shape[0]*Train_data_X_arr.shape[2])

print(Train_data_X_arr[1][0]);

[0. 0. 1.]


In [12]:
print(Train_data_X_reshaped.shape)

print(Train_data_X_reshaped[0])

(165, 11655)
[0. 0. 1. ... 1. 0. 0.]


In [13]:
no_of_gene_trees=int(Train_data_X_reshaped.shape[1]/3)
three_taxa_seq=Train_data_X_reshaped.shape[0]
mask=[]
for i in range (0,three_taxa_seq):
  arr=Train_data_X_reshaped[i]
  mask_row=[]
  for j in range (0,no_of_gene_trees):
    start=j*3
    sum=0
    
    for k in range(start,start+3):
      sum+=arr[k]
      
    mask_row.append(sum)    
  
  mask.append(mask_row)
np_mask=np.array(mask)
print(np_mask.shape)
print(np_mask[2][:40])
#print(nmsk.shape)
#return nmsk
#print(nmsk[0][1])
#print(nmsk.dtype)

(165, 3885)
[1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0.]


In [0]:
def custom_accuracy_function(mask):
    # mask: ekta binary mask e.g [1,0,1,0,1,1,1]
    # 0 gula missing data represent kore
    def custom_accuracy(y_true, y_predict):
        y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
        y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
        is_same = K.cast(K.equal(
            y_true_labels, y_pred_labels), 'float32')
        num_same = K.cast(K.sum(is_same * mask, axis=1), 'float32')
        lengths = K.cast(K.sum(mask, axis=1), 'float32')
        return K.mean(num_same / lengths, axis=0)
      
    return custom_accuracy

def custom_loss_function(mask):
    def custom_loss(y_true, y_predict):
        y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
        y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
        is_same = K.cast(K.equal(
            y_true_labels, y_pred_labels), 'float32')
        num_same = K.cast(K.sum(is_same * mask, axis=1), 'float32')
        lengths = K.cast(K.sum(mask, axis=1), 'float32')
        return K.cast(1-K.mean(num_same / lengths, axis=0), 'float32')
      
    return custom_loss


In [0]:
from keras.layers import Embedding, Add, concatenate, Input, Softmax
from keras import backend as K
#keras.activations.softmax(x, axis=-1)
from keras.activations import softmax



def get_model(input_size):
  
  Input_layer = Input(batch_shape=(None,input_size), name='main_input')
  
  encoder_layer_1 = Dense(int(input_size//1.5), activation='relu', use_bias=True) (Input_layer)
  encoder_layer_2 = Dense(int(input_size//2), activation='relu', use_bias=True) (encoder_layer_1)
  encoder_layer_3 = Dense(int(input_size//2.5), activation='relu', use_bias=True) (encoder_layer_2)
  
  encoder_layer_4 = Dense(int(input_size//3), activation='relu', use_bias=True) (encoder_layer_3)
  encoder_layer_5 = Dense(int(input_size//4), activation='relu', use_bias=True) (encoder_layer_4)
  decoder_layer_0 = Dense(int(input_size//3), activation='relu', use_bias=True) (encoder_layer_5)
  
  decoder_layer_1 = Dense(int(input_size//2.5),activation='relu',use_bias=True) (decoder_layer_0)
  decoder_layer_2 = Dense(int(input_size//2),activation='relu',use_bias=True) (decoder_layer_1)
  decoder_layer_3 = Dense(int(input_size//1.5),activation='relu',use_bias=True) (decoder_layer_2)
  decoder_layer_4 = Dense(input_size,activation='relu',use_bias=True) (decoder_layer_3)
  
  decoder_output = decoder_layer_4
  
  reshaped = Lambda( lambda x: K.reshape(x, (-1, int(decoder_output.shape[1]//3), 3)))(decoder_output)
  
  print(f'Reshaped Shape = {reshaped.shape}')
  
  final_decoded_layer_output = Softmax(axis=-1)(reshaped)
  
  autoencoder = Model(Input_layer, final_decoded_layer_output)
  encoder  = Model(Input_layer,encoder_layer_4)
  #decoder = Model(encoder_layer_4,final_decoded_layer_output)
  
  return autoencoder,encoder

In [16]:
autoencoder, encoder = get_model(input_size=gene_tree_num*triplet)

autoencoder.summary()

W0804 18:45:01.137099 139634357925760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 18:45:01.144233 139634357925760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 18:45:01.145883 139634357925760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Reshaped Shape = (?, 3885, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 11655)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 7770)              90567120  
_________________________________________________________________
dense_2 (Dense)              (None, 5827)              45281617  
_________________________________________________________________
dense_3 (Dense)              (None, 4662)              27170136  
_________________________________________________________________
dense_4 (Dense)              (None, 3885)              18115755  
_________________________________________________________________
dense_5 (Dense)              (None, 2913)              11319918  
_________________________________________________________________
dense_6 (Dense)              (None, 3885)     

In [17]:
Train_data_X_3D_reshaped = Train_data_X_arr.reshape(Train_data_X_arr.shape[1],Train_data_X_arr.shape[0],Train_data_X_arr.shape[2])

print(Train_data_X_arr.shape)

(3885, 165, 3)


In [18]:

autoencoder.compile(optimizer='adam', 
                   loss='binary_crossentropy',
                   sample_weight_mode='temporal',
                   metrics=[custom_accuracy_function(mask=np_mask)])

W0804 18:45:01.304665 139634357925760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0804 18:45:01.333128 139634357925760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0804 18:45:01.339848 139634357925760 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
input_size = Train_data_X_arr.shape[1]

print(input_size)

165


In [20]:
autoencoder.fit(x=Train_data_X_reshaped,
               y=Train_data_X_3D_reshaped,
               epochs=1000,
               batch_size=input_size,
               sample_weight=np_mask,
               verbose=1)

W0804 18:45:01.792089 139634357925760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1000
165/165 [==============================] - 8s 51ms/step - loss: 0.6366 - custom_accuracy: 0.3660
Epoch 2/1000
165/165 [==============================] - 0s 2ms/step - loss: 0.6333 - custom_accuracy: 0.5079
Epoch 3/1000
165/165 [==============================] - 0s 2ms/step - loss: 1.4907 - custom_accuracy: 0.6017
Epoch 4/1000
165/165 [==============================] - 0s 3ms/step - loss: 0.6363 - custom_accuracy: 0.5044
Epoch 5/1000
165/165 [==============================] - 0s 2ms/step - loss: 0.6364 - custom_accuracy: 0.4365
Epoch 6/1000
165/165 [==============================] - 0s 3ms/step - loss: 0.6356 - custom_accuracy: 0.4535
Epoch 7/1000
165/165 [==============================] - 0s 2ms/step - loss: 0.6249 - custom_accuracy: 0.5048
Epoch 8/1000
165/165 [==============================] - 0s 2ms/step - loss: 0.6692 - custom_accuracy: 0.5886
Epoch 9/1000
165/165 [==============================] - 0s 2ms/step - loss: 0.6361 - custom_accuracy: 0.4787
Epoch 10/1000
165/

In [0]:
y_hat = autoencoder.predict(Train_data_X_reshaped)

In [22]:
print(input_size)

165


In [23]:
print(y_hat.shape)
#print(Train_data_Y_arr.shape)
Y_arr = Train_data_Y_arr.reshape(Train_data_Y_arr.shape[1],Train_data_Y_arr.shape[0],Train_data_Y_arr.shape[2])
#print(Y_arr.shape)
y_hat_reshaped = y_hat.reshape(y_hat.shape[1],y_hat.shape[0],y_hat.shape[2])
print(y_hat_reshaped.shape)
for x in range(0,y_hat_reshaped.shape[0]):

  print(str(Train_data_X_arr[x][1]) + str(y_hat_reshaped[x][1]) + " " + str(Train_data_Y_arr[x][1]))

(165, 3885, 3)
(3885, 165, 3)
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 0. 0.][0. 1. 0.] [0. 1. 0.]
[0. 0. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 0. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0.33333334 0.33333334 0.33333334] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0.33333334 0.33333334 0.33333334] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0.33333334 0.33333334 0.33333334] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0.33333334 0.33333334 0.33333334] [0. 1. 0.]
[0. 0. 0.][0. 1. 0.] [0. 1. 0.]
[0. 0. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.][0. 1. 0.] [0. 1. 0.]
[0. 1. 0.]

In [24]:
imputed_mask_position = 1 - np_mask
print(imputed_mask_position.shape)
imputed_mask_position = imputed_mask_position.reshape(imputed_mask_position.shape[1],imputed_mask_position.shape[0])
print(imputed_mask_position.shape)

(165, 3885)
(3885, 165)


In [0]:
def direct_similarity_check_for_imputation(sample_no,Y_tree, Y_hat_tree, imputed_position_mask_for_tree, verbose=0):
#   ck = 0
  mismatch_count = 0
  print("checking tree ->" + str(sample_no))
  #print(np.sum(imputed_position_mask_for_tree))
  for i in range(len(Y_hat_tree)):
    
    if (imputed_position_mask_for_tree[i]==1):
      if(verbose == 0):
        print(i, '>', Y_hat_tree[i], Y_tree[i], imputed_position_mask_for_tree[i])
      if(np.argmax(Y_hat_tree[i]) != np.argmax(Y_tree[i])):
        #print('mismatch at:', i, Y_hat_tree[i], Y_tree[i])
        mismatch_count += 1
    else:
      if(verbose == 0):
        print(i, '>', Y_hat_tree[i], Y_tree[i], imputed_position_mask_for_tree[i])
  #       ck = 1
  #   if ck == 0:
  #     print('>>Fully matched.')
  #     return True
  #   return False
  return mismatch_count

In [26]:
mismatch_count = 0
temp = 0
v_count = 0
y_pred = y_hat_reshaped
for test_sample in range(Train_data_X_arr.shape[0]):
  #print('sample:', test_sample)
  temp = direct_similarity_check_for_imputation(test_sample,Train_data_Y_arr[test_sample], y_pred[test_sample], imputed_position_mask_for_tree=imputed_mask_position[test_sample], verbose=v_count)
  v_count += 1
  mismatch_count += temp
  print("number of mismatch in this tree -> " + str(temp))
  
print(mismatch_count)

temp2 = np.sum(imputed_mask_position)
print(temp2)  
#acc = (1-mismatch_count/(np.size(Train_data_Y_arr,1)*np.size(Train_data_Y_arr,0)))
acc = (1-mismatch_count/temp2)
percent_acc =  acc*100
print(str(percent_acc) + "%")

checking tree ->0
0 > [0.33333334 0.33333334 0.33333334] [0. 0. 1.] 0.0
1 > [0. 1. 0.] [0. 1. 0.] 0.0
2 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
3 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
4 > [1. 0. 0.] [1. 0. 0.] 0.0
5 > [1. 0. 0.] [1. 0. 0.] 0.0
6 > [1. 0. 0.] [1. 0. 0.] 0.0
7 > [1. 0. 0.] [1. 0. 0.] 0.0
8 > [1. 0. 0.] [1. 0. 0.] 0.0
9 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
10 > [0. 1. 0.] [0. 1. 0.] 0.0
11 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
12 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
13 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
14 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
15 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
16 > [0. 1. 0.] [0. 1. 0.] 0.0
17 > [0.33333334 0.33333334 0.33333334] [0. 1. 0.] 0.0
18 > [0.33333334 0.33333334 0.33333334] [0. 0. 1.] 0.0
19 > [0. 0. 1.] [0. 0. 1.] 0.0
20 > [0. 0. 1.] [0. 0. 1.] 0.0
21 > [0. 0. 1.] [0. 0. 1.] 0.0
22 > [0. 0. 1.] [0. 0. 1.] 0.0
23 > [0.3333333